In [134]:
import os
import pandas as pd
import numpy as np
import csv
from pandas import DataFrame
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split, GridSearchCV
from keras import layers
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import svm   
from imblearn.under_sampling import RandomUnderSampler


In [126]:
data = pd.read_csv("train_file.csv")
data1 = pd.read_csv("test_file.csv")
Train_features = pd.read_csv("Training.csv")
Test_features = pd.read_csv("Test.csv")
ros = RandomOverSampler(random_state=9)
X_sample_0, y_sample_0 = ros.fit_sample(Train_features, data['Category'])
X_train_O,X_test_O,y_train_O,y_test_O = train_test_split(X_sample_0,y_sample_0,test_size = 0.3)

In [144]:
rus = RandomUnderSampler(random_state=8)
X_sample_u, y_sample_u =  rus.fit_sample(Train_features, data['Category'])
X_train_u,X_test_u,y_train_u,y_test_u = train_test_split(X_sample_u,y_sample_u,test_size = 0.3)    

In [146]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [147]:
svc_param_selection(X_train_u, y_train_u, 2)

{'C': 10, 'gamma': 0.01}

In [151]:
rbf = svm.SVC(kernel='rbf', gamma=0.01, C=0.01).fit(X_train_O, y_train_O)
y_pred = rbf.predict(X_test_O)
f1_score(y_test_O, y_pred, average='weighted')

c:\users\venkatachalam s\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.06506217244561886

In [133]:
svm =  rbf.predict(Test_features.as_matrix())
svm_prediction = pd.concat([data1['Application/Permit Number'],pd.DataFrame({'Category':svm})],axis=1)
svm_prediction.to_csv('prediction12.csv', sep=',',index = False)

In [127]:
model = XGBClassifier(n_estimators=15000,n_jobs=-1)
model.fit(X_train_O, y_train_O)
y_pred = model.predict(X_test_O)
f1_score(y_test_O, y_pred, average='weighted')

0.9650336997589892

In [128]:
Boosting =  model.predict(Test_features.as_matrix())
Boosting_prediction = pd.concat([data1['Application/Permit Number'],pd.DataFrame({'Category':Boosting})],axis=1)
Boosting_prediction.to_csv('prediction12.csv', sep=',',index = False)

In [44]:
# Deep learning model
def keras_model():
    model1 = Sequential()
    model1.add(Dense(250, input_dim = np.int(X_train_O.shape[1]),activation ='relu'))
    model1.add(Dense(250,activation ='relu'))
    model1.add(Dropout(0.5))
    model1.add(Dense(250,activation ='relu'))
    model1.add(Dropout(0.5))
    model1.add(Dense(250,activation ='relu'))
    model1.add(Dense(5,activation='softmax'))
    model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model1
encoder = LabelEncoder()
encoder.fit(y_sample_0)
encoded_Y = encoder.transform(y_sample_0)
dummy_y = np_utils.to_categorical(encoded_Y)
estimator = KerasClassifier(build_fn=keras_model, epochs=1, batch_size=5, verbose=0)
kfold = KFold(n_splits=3, shuffle=True, random_state=50)
results = cross_val_score(estimator, X_sample_0, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [129]:
pred1 = pd.read_csv("prediction1.csv")
pred2 = pd.read_csv("prediction2.csv")
pred3 = pd.read_csv("prediction3.csv")
pred4 = pd.read_csv("prediction4.csv")
pred5 = pd.read_csv("prediction5.csv")
pred6 = pd.read_csv("prediction10.csv")
pred7 = pd.read_csv("prediction7.csv")
final_pred = pd.concat([pred2['Category'],
                       pred2['Category'],
                       pred3['Category'],
                       pred4['Category'],
                       pred5['Category'],
                       pred6['Category'],
                       pred7['Category']],axis=1)
labels = []
for i in range(len(final_pred)):
    labels.append(Counter(final_pred.loc[i]).most_common(1)[0][0])

In [120]:
Max_prediction['Category'].value_counts()

SINGLE FAMILY / DUPLEX    12010
COMMERCIAL                 6004
MULTIFAMILY                3114
INSTITUTIONAL               740
INDUSTRIAL                  492
Name: Category, dtype: int64

In [130]:
Max_prediction = pd.concat([data1['Application/Permit Number'],pd.DataFrame({'Category':labels})],axis=1)
Max_prediction.to_csv('prediction11.csv', sep=',',index = False)
